In [ ]:
import requests
import tabula
import pandas as pd

# Download of Module Handbooks

In [ ]:
pdf_url_bachelor_business_management_and_economics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-184-H-2008.pdf"
pdf_url_bachelor_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-277-H-2021.pdf"
pdf_url_bachelor_economathematics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-276-H-2015.pdf"
pdf_url_master_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-j10-H-2019.pdf"
pdf_url_master_management ="https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-i45-H-2018.pdf"
pdf_url_master_international_economic_policy = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-h92-H-2018.pdf"

response = requests.get(pdf_url_bachelor_business_management_and_economics)
with open("module_handbooks/bachelor_business_management_and_economics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_business_management_and_economics.pdf")

response = requests.get(pdf_url_bachelor_information_systems)
with open("module_handbooks/bachelor_information_systems.pdf", "wb") as f:
    f.write(response.content)

print("Downloaded bachelor_information_systems.pdf")
response = requests.get(pdf_url_bachelor_economathematics)
with open("module_handbooks/bachelor_economathematics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_economathematics.pdf")

response = requests.get(pdf_url_master_information_systems)
with open("module_handbooks/master_information_systems.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_information_systems.pdf")

response = requests.get(pdf_url_master_management)
with open("module_handbooks/master_management.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_management.pdf")

response = requests.get(pdf_url_master_international_economic_policy)
with open("module_handbooks/master_international_economic_policy.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_international_economic_policy.pdf")

# Ansatz über TABULA --> Verforfen 
Reading Tables from Module Book for master_information_systems

In [ ]:
import pandas as pd
import tabula
import re

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:


tables_master_is = tabula.read_pdf("module_handbooks/master_information_systems.pdf", pages='10-30', multiple_tables=True)
print(len(tables_master_is))
print(tables_master_is)

# PDF Miner Ansatz wie Felix -> über Regex

In [ ]:
import io
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage


def pdf_to_text(file_path, n=0):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    laparams = LAParams()
    converter = TextConverter(
        resource_manager,
        fake_file_handle,
        codec="utf-8",
        laparams=laparams,
    )
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(file_path, "rb") as fh:
        for i, page in enumerate(PDFPage.get_pages(fh, caching=True, check_extractable=True)):
            if i < n:
                continue
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    # close open handles
    converter.close()
    fake_file_handle.close()

    if text:
        return remove_header_footer(text)
    else:
        return ""


def remove_header_footer(text):
    # Split the text into lines
    lines = text.split("\n")

    # Remove lines that match the header pattern
    lines = [line for line in lines if not line.startswith("JMU Würzburg")]

    # Remove lines that match the footer pattern
    lines = [line for line in lines if not line.startswith("page")]
    lines = [line for line in lines if not re.match(r".*Master.* with 1 major Information Systems.*", line)]
    lines = [line for line in lines if not re.match(r".*Master.* Information Systems - 2019.*", line)]
    lines = [line for line in lines if not re.match(r".*Module Catalogue for the Subject.*", line)]
    lines = [line for line in lines if not re.match(r".*Information Systems.*", line)]
    lines = [line for line in lines if not re.match(r".*Master’s with 1 major, 120 ECTS credits.*", line)]


    # Join the remaining lines back into a single string
    clean_text = "\n".join(lines)

    return clean_text


In [ ]:
# define a function which extracts the 

In [ ]:
text = pdf_to_text("module_handbooks/master_information_systems.pdf", n=9)


In [ ]:
text_no_footer_header = remove_header_footer(text)
print(text_no_footer_header)

with open("module_handbooks/master_information_systems_processed.txt", "w") as f:
    f.write(text_no_footer_header)

In [ ]:
tables = re.split(r"(?=Module title)", text_no_footer_header)[1:]
print(len(tables))
print("---------------TABLE 0--------------------------")
print(tables[0])
# store to table.txt file 
with open("module_handbooks/master_information_systems_table_0.txt", "w") as f:
    f.write(tables[0])

print("---------------TABLE 1--------------------------")
print(tables[1])

In [114]:
def extract_table_values(table_string):
    # Define regex patterns
    patterns = {
        "Module Title": r"Module title\n\n(.+)",
        "Abbreviation": r"Abbreviation\n\n(.+)",
        "Module Coordinator": r"Module coordinator\n\n(.+)",
        "Module Offered By": r"Module offered by\n\n(.+)",
        "ECTS": r"ECTS Method of grading\n\n(.+)",
        "Method of Grading": r"Method of grading\n\n(.+)",
        "Completion of Modules": r"Only after succ\. compl\. of module\(s\)\n\n(.+)",
        "Duration": r"Duration\n\n(.+)",
        "Module Level": r"Module level\n\n(.+)",
        "Other Prerequisites": r"Other prerequisites\n\n(.+)",
        "Contents": r"Contents\n\n(.+)",
        "Intended Learning Outcomes": r"Intended learning outcomes\n\n(.+)",
        "Courses": r"Courses \(type, number of weekly contact hours, language — if other than German\)\n\n(.+)",
        "Method of Assessment": r"Method of assessment \(type, scope, language — if other than German, examination offered — if not every semester, information on whether module is creditable for bonus\)\n\n(.+)",
        "Allocation of Places": r"Allocation of places\n\n(.+)",
        "Additional Information": r"Additional information\n\n(.+)",
        "Workload": r"Workload\n\n(.+)",
        "Teaching Cycle": r"Teaching cycle\n\n(.+)",
        "Referred to in LPO I": r"Referred to in LPO I  \(examination regulations for teaching-degree programmes\)\n\n(.+)",
        "Page": r"Page\n\n(.+)"
    }
    
    extracted_values = {}
    
    # Extract values using regex
    for key, pattern in patterns.items():
        match = re.search(pattern, table_string)
        if match:
            extracted_values[key] = match.group(1)
    
    return extracted_values


In [115]:
values = extract_table_values(tables[0])
print(values)

# Print the extracted values
for key, value in values.items():
    print(key + ": " + value)

{'Module Title': 'Information Processing within Organizations', 'Abbreviation': '12-IV-161-m01', 'Module Coordinator': 'Module offered by', 'Module Offered By': 'holder of the Chair of Business Management and Business ', 'ECTS': 'Only after succ. compl. of module(s)', 'Method of Grading': 'Only after succ. compl. of module(s)', 'Completion of Modules': '5', 'Duration': 'Module level', 'Module Level': 'Other prerequisites', 'Other Prerequisites': '1 semester', 'Contents': 'Content:', 'Intended Learning Outcomes': 'After completing the course "Integrated Information Processing", students will be able to', 'Courses': 'V (2) + Ü (2)', 'Allocation of Places': '--', 'Additional Information': '--', 'Workload': '150 h', 'Teaching Cycle': '--', 'Referred to in LPO I': '--'}
Module Title: Information Processing within Organizations
Abbreviation: 12-IV-161-m01
Module Coordinator: Module offered by
Module Offered By: holder of the Chair of Business Management and Business 
ECTS: Only after succ. c

# Create final Dataframe with these columns

In [ ]:
module_handbook_columns = [
    "module title",
    "abbreviation",
    "module coordinator",
    "module offered by",
    "ects",
    "method of grading",
    "only after successfull completion of module(s)",
    "duration",
    "module level",
    "other prerequisites",
    "contents",
    "intended learning outcomes",
    "courses",
    "method of assessment",
    "allocation of places",
    "additional information",
    "workload",
    "teaching cycle",
    "referred to in lpo I",
    "page",
]

module_handbook_df = pd.DataFrame(columns=module_handbook_columns)
print(module_handbook_df)
print(module_handbook_df.shape)
module_handbook_df.to_csv(
    "module_handbooks/master_information_systems.csv", index=False
)